  Using cached tensorflow-1.4.0-cp36-cp36m-win_amd64.whl
  Found existing installation: html5lib 0.999
    Uninstalling html5lib-0.999:
      Successfully uninstalled html5lib-0.999


    DEPRECATION: Uninstalling a distutils installed project (html5lib) has been deprecated and will be removed in a future version. This is due to the fact that uninstalling a distutils project will only partially uninstall the project.


In [1]:
#importing libraries--------------------
import pandas as pd
from keras.layers import Dense, Input, Flatten, Dropout, Lambda
from keras.models import Model, Sequential
from sklearn.feature_extraction.text import TfidfVectorizer 
from keras.utils.np_utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [2]:
#importing libraries--------------------
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from keras.utils.np_utils import to_categorical

In [3]:
#processing raw dataset and converting catogorical labels to numerical one.....................
print('Processing text dataset')
raw_data = pd.read_csv('RAW_DATA.csv', encoding='latin-1')
labels_index = {}
for idx, i in enumerate(raw_data.CATEGORIES.unique()):
    labels_index[i] = idx

Processing text dataset


In [4]:
#converting catogorical labels to numerical one.....................
print('Found %s texts.' % len(raw_data.SUMMARY))
print('Found  %s  categories.' % len(labels_index))

print('Column Names', raw_data.columns)
labels = [labels_index[label] for label in raw_data.CATEGORIES]
print(labels)
#labels = to_categorical(np.asarray(labels))


Found 57280 texts.
Found  21  categories.
Column Names Index(['fileid', 'SUMMARY', 'DATA', 'CATEGORIES', 'previous_appointment',
       'ID'],
      dtype='object')
[0, 1, 1, 2, 3, 4, 5, 3, 6, 2, 2, 7, 8, 7, 7, 5, 7, 3, 9, 0, 2, 1, 7, 10, 11, 10, 6, 4, 2, 0, 0, 7, 12, 2, 0, 3, 7, 5, 10, 4, 0, 7, 9, 10, 2, 8, 0, 0, 2, 0, 0, 2, 7, 3, 2, 2, 13, 14, 1, 0, 5, 7, 12, 7, 2, 2, 1, 0, 2, 1, 7, 7, 8, 1, 12, 0, 0, 5, 6, 0, 1, 1, 3, 5, 5, 1, 5, 0, 2, 2, 2, 1, 7, 0, 2, 7, 11, 0, 0, 11, 0, 13, 2, 0, 0, 5, 10, 1, 1, 2, 5, 1, 0, 14, 0, 1, 2, 0, 0, 2, 0, 2, 7, 1, 3, 7, 5, 7, 1, 7, 3, 2, 5, 0, 8, 7, 2, 1, 12, 7, 5, 12, 9, 0, 0, 3, 15, 1, 0, 8, 0, 2, 1, 7, 1, 0, 2, 1, 7, 2, 2, 7, 11, 1, 1, 2, 2, 5, 1, 1, 14, 8, 5, 2, 7, 0, 8, 7, 7, 3, 5, 2, 2, 0, 2, 3, 12, 0, 1, 7, 6, 2, 1, 1, 1, 1, 10, 1, 0, 3, 1, 7, 7, 7, 0, 0, 0, 7, 0, 0, 0, 1, 1, 0, 9, 3, 7, 13, 3, 0, 8, 2, 1, 8, 8, 0, 0, 0, 15, 0, 0, 2, 0, 1, 8, 10, 10, 0, 7, 1, 1, 7, 5, 7, 0, 2, 7, 12, 0, 7, 1, 0, 10, 1, 0, 1, 7, 7, 7, 14, 7, 14, 9, 1, 9, 6, 12, 2,

In [5]:
#Importing stopwords for english lang.......................
stopwords = nltk.corpus.stopwords
eng_stopwords = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

In [5]:
#all_text=[]
#def basic_preprocessing(texts):
 #   #print(texts[2])
  #  for i in range(len(texts)):
   #     text = str(texts[i]).lower() #lowering
    #    text = re.sub(r'/|&|@|:|;|,| \.\.+ | \?|\'s|-', ' ', text)
     #   text = word_tokenize(text)
      #  text = [word for word in text if word not in eng_stopwords] #removing stop words
       # text = [wordnet_lemmatizer.lemmatize(word) for word in text]
        #all_text.append(text)
    #return(all_text)
#print(basic_preprocessing(raw_data.SUMMARY))


KeyboardInterrupt: 

In [6]:
#stopword is updated by adding some unwanted words and stoppers.................
all_detailed_text=[]
documents=[]
def basic_preprocessing(texts):
   # print(texts)
    
    for i in range(len(texts)):
        
        res='';
        d=[]
        text = str(texts[i]).lower()
        
        words = word_tokenize(text)
        pattern =r'[\d+]'
        pp=re.compile(pattern)
        Stopwords=set([word for word in words if word.startswith('\\') or '\\' in word or '/' in word or  'xxx' in word or re.search(pp,word)])
        Stopwords.update(('paragraph',';',':','?','$','%','@',',','\'s','#','!','-','--','.','(',')','{','}','[',']','default','call ','phone','additional', 'follow-up', 'details','arial','font','caller', 'call'))
        eng_stopwords.update(Stopwords)
        #pattern=r'\\*'
        #eng_stopwords.update(('{','}',pattern))
        words = [word for word in words if word not in eng_stopwords]
       # print(words)
        documents.append(words)
        all_detailed_text.extend(words)
       # print(text)
        #for m in re.finditer('\par', text):
         #   #print('ll found', m.start(), m.end())
         #   d.append(m.start())
          #  d.append(m.end())
            #print("me")
        #text = re.sub(r'/|&|@|:|;|,| \.\.+ | \?|\'s|-', ' ', text)
        #text = word_tokenize(text)
        #text = [word for word in text if word not in eng_stopwords] #removing stop words
        #text = [wordnet_lemmatizer.lemmatize(word) for word in text]
        #all_text.append(text)
        #print(d)
        #for i in range(1, len(d)):
         #   if d[i]-d[i-1]+1>=50:
          #      res =res+text[d[i-1]:d[i]];
            #print("================")
        #print(res)
    return 0
basic_preprocessing(raw_data.DATA)

0

In [7]:
print(documents[:2])

[['normal', 'note', 'patient', 'back', 'cell', 'name', 'patient', 'returning', 'nurse', 'unable', 'make', 'appt', 'without', 'talking', 'fin', 'service', 'dept', 'however', 'needs', 'medication', 'worried', 'issue', 'without', 'medication', 'please', 'patient', 'discuss', 'taken', 'may', 'pt', 'returned', 'please', 'back', 'advise', 'may', 'problem', 'without', 'insurance', 'non-compliant', 'instructions', 'come', 'appt', 'refills', 'without', 'david', 'may', 'rn', 'spoke', 'pt', 'relayed', 'requested', 'speak', 'financial', 'services', 'rn', 'transferred', 'business', 'office', 'rn', 'requested', 'business', 'office', 'matter', 'hollie', 'saltis', 'rn', 'may', 'david', 'may'], ['normal', 'note', 'patient', 'back', 'cell', 'name', 'pt', 'school', 'teacher', 'reporting', 'pt', 'able', 'sit', 'still', 'mom', 'wants', 'know', 'focalin', 'needs', 'dosage', 'adjusting', 'something', 'pt', 'could', 'take', 'school', 'staff', 'could', 'administer', 'please', 'back', 'discuss', 'taken', 'may',

In [8]:
# for tf-idf list document is converted into string 
# training and testting data is splitting into .....
#total size of dataste =57280
# training data size= 50000
# testing data size= 7280

tf_idf_text=[]
for document in documents:
    tf_idf_text.append(' '.join(str(e) for e in document))
train_tf_idf_text=tf_idf_text[:50000]
test_tf_idf_text=tf_idf_text[50000:]
print(len(train_tf_idf_text))
print(len(test_tf_idf_text))




50000
7280


In [9]:
from nltk import FreqDist
freqs = FreqDist(all_detailed_text)
print(freqs)
Dict=freqs.most_common(2000) # creating vocab size of 2000 words.........
all_words=[]
for i in Dict:
    all_words.append(i[0])
print(all_words)

<FreqDist with 34927 samples and 3052418 outcomes>
['pt', 'patient', 'normal', 'back', 'taken', 'note', 'rn', 'mg', 'called', 'rna', 'po', 'please', 'rx', 'tabs', 'may', 'cell', 'action', 'clinical', 'mom', 'name', 'march', 'list', 'april', 'md', 'appt', 'would', 'x', 'message', 'january', 'home', 'june', 'entered', 'july', 'wrote', 'prescription', 'medication', 'signed', 'states', 'like', 'february', 'hcl', 'fax', 'new', 'medications', 'tab', 'updated', 'mri', 'spoke', 'last', 'see', 'know', 'pharmacy', 'dr.', 'refill', 'pain', 'prn', 'needs', 'sent', 'pm', 'take', 'give', 'advise', 'order', 'lpn', 'need', "n't", 'understanding', 'results', 'time', 'appointments', 'get', 'verbalized', 'completed', 'schedule', 'bid', 'reason', 'print', 'flag', 'says', 'converted', 'dr', 'per', 'scheduled', 'provider', 'office', 'today', 'wants', 'also', 'q', 'electronically', 'used', 'nc', 'lisa', 'ph', 'rov', 'pls', 'fill', 'one', 'medfusion', 'faxed', 'work', 'hydrocodone-acetaminophen', 'oral', 'day

In [14]:
def get_featureset(document): # feature set is creating using (Bag Of Words)BOW model................
    return({word: (word in document) for word in all_words})
processed_texts = [get_featureset(document) for document in documents]

In [15]:
print(len(processed_texts),len(labels))

57280 57280


In [16]:
train_data = list(zip(processed_texts, labels)) #  data set and label converted into one...............

In [17]:
print(len(train_data))

57280


In [18]:
print(type(train_data[0][0]))

<class 'dict'>


In [19]:
import random
random.shuffle(train_data) # Shuffling of dataset so that can't have any bias set of instances in training and testing dataset.....
print(len(train_data))
print(train_data[1])

57280
({'pt': False, 'patient': False, 'normal': True, 'back': False, 'taken': False, 'note': False, 'rn': False, 'mg': False, 'called': False, 'rna': False, 'po': False, 'please': True, 'rx': False, 'tabs': False, 'may': False, 'cell': False, 'action': False, 'clinical': False, 'mom': False, 'name': False, 'march': False, 'list': False, 'april': False, 'md': True, 'appt': True, 'would': False, 'x': False, 'message': False, 'january': False, 'home': True, 'june': False, 'entered': False, 'july': False, 'wrote': True, 'prescription': False, 'medication': False, 'signed': False, 'states': False, 'like': False, 'february': False, 'hcl': False, 'fax': False, 'new': False, 'medications': False, 'tab': False, 'updated': False, 'mri': False, 'spoke': False, 'last': False, 'see': False, 'know': False, 'pharmacy': False, 'dr.': False, 'refill': False, 'pain': False, 'prn': False, 'needs': False, 'sent': False, 'pm': True, 'take': False, 'give': False, 'advise': False, 'order': False, 'lpn': Fal

In [20]:
train_target=[]
train_Data=[] # consist only feautres not labels....
data=train_data;# consist training labels......
train_data=train_data[:50000] # training data set have 50000 instances ....
for i in train_data:
    train_Data.append(i[0])
    train_target.append(i[1])
test_data=data[50000:]
print(len(data))
print(len(train_data))
print(len(test_data))

57280
50000
7280


In [21]:
test_target=[]#consist test labels......
test_Data=[]# consist only feautres not labels....
for i in test_data:
    test_Data.append(i[0])
    test_target.append(i[1])
print(test_target[:39])
print(type(test_Data[0]))

[7, 7, 12, 2, 1, 0, 7, 7, 11, 7, 2, 8, 7, 0, 10, 12, 7, 0, 12, 0, 2, 10, 0, 6, 2, 11, 14, 0, 0, 2, 7, 1, 0, 1, 0, 0, 7, 2, 0]
<class 'dict'>


In [19]:
classifier = nltk.NaiveBayesClassifier.train(train_data) # trining Naive Baye's classifier on BOW 
classifier.show_most_informative_features()

Most Informative Features
                    rxid = True                0 : 7      =   1689.5 : 1.0
                 traffic = True               15 : 1      =   1156.0 : 1.0
                 walmart = True               19 : 7      =    881.2 : 1.0
              authorized = True                0 : 12     =    660.0 : 1.0
                   print = True                0 : 7      =    651.0 : 1.0
                    elig = True               11 : 0      =    648.9 : 1.0
                    peer = True               16 : 1      =    640.4 : 1.0
                  retail = True               19 : 2      =    619.2 : 1.0
                   admin = True                8 : 0      =    605.1 : 1.0
                 ncpdpid = True               19 : 7      =    528.8 : 1.0


In [31]:
test_predictions = classifier.classify_many(test_Data)    #prediciting test data labels........... 
print(test_predictions)

[10, 0, 7, 0, 7, 11, 7, 17, 11, 0, 2, 2, 9, 0, 7, 0, 5, 2, 12, 7, 0, 7, 1, 0, 10, 1, 2, 2, 5, 14, 14, 8, 0, 2, 6, 8, 2, 7, 2, 7, 8, 18, 0, 7, 8, 7, 5, 7, 7, 6, 7, 12, 1, 12, 7, 0, 10, 0, 13, 1, 1, 2, 12, 14, 4, 7, 7, 2, 1, 6, 0, 5, 4, 12, 9, 7, 12, 8, 7, 7, 7, 7, 14, 3, 7, 3, 7, 1, 11, 7, 0, 0, 9, 1, 1, 8, 0, 9, 7, 0, 0, 7, 6, 0, 10, 6, 7, 2, 3, 2, 4, 0, 4, 0, 7, 4, 0, 2, 7, 1, 4, 1, 6, 3, 5, 1, 1, 0, 4, 0, 7, 1, 3, 0, 0, 5, 12, 11, 7, 0, 7, 7, 1, 15, 4, 10, 0, 9, 6, 9, 12, 8, 9, 0, 4, 12, 1, 4, 1, 2, 7, 12, 1, 5, 0, 5, 8, 14, 7, 5, 12, 0, 3, 0, 7, 7, 7, 7, 7, 7, 13, 5, 9, 9, 5, 1, 3, 7, 1, 10, 7, 2, 15, 12, 1, 7, 2, 7, 0, 8, 15, 3, 7, 1, 0, 7, 9, 11, 12, 7, 14, 0, 0, 7, 6, 11, 7, 0, 6, 1, 1, 0, 0, 5, 0, 15, 3, 6, 7, 6, 3, 7, 12, 7, 2, 0, 1, 5, 1, 9, 14, 7, 0, 3, 8, 1, 0, 1, 11, 12, 8, 7, 3, 7, 1, 7, 11, 7, 4, 2, 3, 7, 1, 1, 5, 0, 7, 14, 4, 8, 12, 7, 3, 1, 8, 8, 0, 10, 5, 4, 7, 0, 18, 12, 18, 7, 8, 15, 1, 8, 7, 1, 7, 7, 1, 7, 7, 0, 7, 3, 2, 0, 8, 1, 10, 0, 5, 4, 4, 1, 7, 0, 0, 5, 7, 7,

In [32]:
from sklearn.metrics import accuracy_score  # accuracy for naive baye's on naive Baye's ........

#backslash means the function continues in next line\
print('Accuracy on test data: {}%'.format(\
                                          accuracy_score(test_target, test_predictions)\
                                          *100))

Accuracy on test data: 53.214285714285715%


In [33]:
#creating data for dt # creating training and test dataset for decision tree because it does not accept dictionary as input.....
dt_train_data=[]   
dt_test_data=[]

for i in train_Data:
    dt_train_data.append(list(i.values()))
    
for i in test_Data:
    dt_test_data.append(list(i.values()))    

In [34]:
print(type(dt_train_data[0]))
print(type(dt_test_data[0]))
print(len(dt_train_data))
print(len(dt_test_data))

<class 'list'>
<class 'list'>
50000
7280


In [35]:
print(type(train_Data[0]))
print(type(train_target))
print(type(test_Data[0]))
print(type(test_target))



<class 'dict'>
<class 'list'>
<class 'dict'>
<class 'list'>


In [ ]:
#this is for decision tree on bow

In [27]:
from sklearn import tree

In [28]:
model = tree.DecisionTreeClassifier(criterion='gini') # for classification, here you can change the algorithm as gini or entropy (information gain) by default it is gini  
# model = tree.DecisionTreeRegressor() for regression
# Train the model using the training sets and check score


In [29]:
model.fit(dt_train_data, train_target)
model.score(dt_train_data, train_target) # training Decision tree classifier on BOW.....


0.98972000000000004

In [30]:
#Predict Output
predicted= model.predict(dt_test_data)  # prediciting test dataset.....


In [33]:
from sklearn.metrics import accuracy_score # getting accuaracy of decision tree using BOW ....
print('Accuracy on test data: {}%'.format(\
                                          accuracy_score(test_target, predicted)\
                                          *100))

Accuracy on test data: 58.269230769230774%


In [34]:
from sklearn.ensemble import RandomForestClassifier # random forest on BOW.....

In [46]:
model= RandomForestClassifier(n_estimators=200)

In [47]:
model.fit(dt_train_data, train_target) #training of random forest..........
model.score(dt_train_data, train_target)

0.98972000000000004

In [48]:
predicted= model.predict(dt_test_data) # predicting on ddataset of random forest using BOW.....


In [49]:
print('Accuracy on test data: {}%'.format(\ # acuuracy of random forest using BOW.....
                                          accuracy_score(test_target, predicted)\
                                          *100))

Accuracy on test data: 67.07417582417582%


In [50]:
#rom sklearn.ensemble import GradientBoostingClassifier 

In [61]:
#clf = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=1)


In [62]:
#clf.fit(dt_train_data, train_target)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=1.0, loss='deviance', max_depth=1,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [63]:
#clf.score(dt_train_data, train_target)

0.034079999999999999

In [66]:
#predicted= clf.predict(dt_test_data)

In [67]:
#print('Accuracy on test data: {}%'.format(\
                                          accuracy_score(test_target, predicted)\
                                          *100))

Accuracy on test data: 3.6813186813186816%


In [ ]:
#logistic regression

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [70]:
logreg_classifier = LogisticRegression() # logistic regression using BOW.......



In [71]:
logreg_classifier.fit(dt_train_data, train_target) # training...........

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [72]:
train_predictions = logreg_classifier.predict(dt_train_data) #prediction..........
test_predictions = logreg_classifier.predict(dt_test_data)



In [73]:
print('Train Accuracy: {}%'.format(accuracy_score(train_predictions, train_target)*100))
print('Test Accuracy: {}%'.format(accuracy_score(test_predictions, test_target)*100)) # training and test accuracy
                                                                                      #of logistic regression using BOW


Train Accuracy: 76.408%
Test Accuracy: 67.82967032967034%


In [ ]:
#svm

In [36]:
from sklearn.svm import SVC

svm_classifier = SVC() # SVM using BOW.......



In [37]:
svm_classifier.fit(dt_train_data, train_target)#training..........

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
train_predictions = svm_classifier.predict(dt_train_data)
print('Train Accuracy: {}%'.format(accuracy_score(train_predictions, train_target)*100))# accuracy on training data of SVM using BOW......

In [38]:
test_predictions = svm_classifier.predict(dt_test_data)# prediction........

print('Test Accuracy: {}%'.format(accuracy_score(test_predictions, test_target)*100))# accuracy on test data of SVM using BOW......

Test Accuracy: 59.010989010989015%


In [ ]:
#knn

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier 

#knn = KNeighborsClassifier(n_neighbors=5)



In [ ]:
#knn.fit(dt_train_data, train_target)

In [ ]:
#test_predictions = knn.predict(dt_test_data)
#print('Test Accuracy: {}%'.format(accuracy_score(test_predictions, test_target)*100))


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()  # creating tf-idf vector as a feature set of documents......
tfidf.fit(train_tf_idf_text) 
train_tfidf = tfidf.transform(train_tf_idf_text)

In [63]:
print(train_tfidf.shape)

(50000, 28396)


In [ ]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import accuracy_score

#logreg_classifier = LogisticRegression() 
#logreg_classifier.fit(train_tfidf, train_target)

#train_predictions = logreg_classifier.predict(train_tfidf)
#test_predictions = logreg_classifier.predict(test_tfidf)

#print('Train Accuracy: {}%'.format(accuracy_score(train_predictions, train_target)*100))
#print('Test Accuracy: {}%'.format(accuracy_score(test_predictions, test_target)*100))


In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg_classifier = LogisticRegression() # logistic regression using tf-idf as embedding ......
logreg_classifier.fit(train_tfidf, train_target) #training logistic regression......



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
test_tfidf = tfidf.transform(test_tf_idf_text) # predicting logistic regression....

In [57]:
train_predictions = logreg_classifier.predict(train_tfidf)
test_predictions = logreg_classifier.predict(test_tfidf) # prediction of logistic regression using tf-idf on test data.....



In [58]:
print('Train Accuracy: {}%'.format(accuracy_score(train_predictions, train_target)*100)) #train accuracy......


Train Accuracy: 32.51%


In [59]:
print('Test Accuracy: {}%'.format(accuracy_score(test_predictions, test_target)*100)) #test accuracy.......

Test Accuracy: 18.365384615384617%


In [23]:
from sklearn.svm import SVC

svm_classifier = SVC()     #support vector machine (SVM) using tf-idf .......




In [24]:
svm_classifier.fit(train_tfidf, train_target) #training 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
train_predictions = svm_classifier.predict(train_tfidf)

In [ ]:

print('Train Accuracy: {}%'.format(accuracy_score(train_predictions, train_target)*100))

In [28]:

test_predictions = svm_classifier.predict(test_tfidf) #test prediction......


In [30]:
from sklearn.metrics import accuracy_score
print('Test Accuracy: {}%'.format(accuracy_score(test_predictions, test_target)*100)) # svm accuracy using tf-idf as word embedding ...

Test Accuracy: 18.42032967032967%


In [63]:
#from sklearn.neighbors import KNeighborsClassifier

#knn = KNeighborsClassifier(n_neighbors=5)
#knn.fit(train_tfidf, train_target)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [64]:
#test_predictions = knn.predict(test_tfidf)
#print('Test Accuracy: {}%'.format(accuracy_score(test_predictions, test_target)*100))


MemoryError: 

In [39]:
#dt on tf-idf.....


from sklearn import tree   #decision tree using tf-idf as word embedding ........

In [40]:
model = tree.DecisionTreeClassifier(criterion='gini') # for classification, here you can change the algorithm as gini or entropy (information gain) by default it is gini  
# model = tree.DecisionTreeRegressor() for regression
# Train the model using the training sets and check score

In [41]:
model.fit(train_tfidf, train_target)
model.score(train_tfidf, train_target) # training ......


0.94832000000000005

In [42]:
#Predict Output
predicted= model.predict(test_tfidf) # testing or prediction..... 

In [43]:
from sklearn.metrics import accuracy_score # accuracy of decision treee using tf-idf as word embedding on test data.....
print('Accuracy on test data: {}%'.format(\
                                          accuracy_score(test_target, predicted)\
                                          *100))

Accuracy on test data: 13.214285714285715%


In [44]:
#rf on tf-idf..........

from sklearn.ensemble import RandomForestClassifier  # random forest for tf-idf......

In [45]:
model= RandomForestClassifier(n_estimators=200)

In [46]:
model.fit(train_tfidf, train_target) # training,,,../......
model.score(train_tfidf, train_target)

0.94832000000000005

In [48]:
predicted= model.predict(test_tfidf) # testing..........


In [49]:
print('Accuracy on test data: {}%'.format(\ # accuracy of random forest using tf-idf as word embeeding  on test data.....
                                          accuracy_score(test_target, predicted)\
                                          *100))

Accuracy on test data: 17.197802197802197%


In [50]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Lambda    # importing libraries for neural network.........
from keras.layers import Embedding
from keras.models import Sequential
import tensorflow as tf
import pandas as pd
import os

In [51]:
BASE_DIR = os.getcwd()
GLOVE_DIR = BASE_DIR
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [52]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]                                 # retriving embedding for words from pre-trained word vector model....... 
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found {} word vectors.'.format(len(embeddings_index)))


Indexing word vectors.
Found 400000 word vectors.


In [53]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(tf_idf_text)
sequences = tokenizer.texts_to_sequences(tf_idf_text)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)


Found 30737 unique tokens.


In [54]:


labels = to_categorical(np.asarray(labels)) # converting target variable to catagorical variable ....
print(len(labels))   
print(labels)


57280
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]]


In [55]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (57280, 1000)
Shape of label tensor: (57280, 21)


In [56]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0]) # creating validation split.......

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [57]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing embedding matrix.


In [58]:
print('Training model.')

model = Sequential() # creating neural network layer.........
model.add(Embedding(num_words,
                    EMBEDDING_DIM,
                    input_length=MAX_SEQUENCE_LENGTH,
                    weights=[embedding_matrix],
#                     trainable=False
                    ))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(500, activation = 'relu'))
model.add(Dense(len(labels_index), activation='softmax'))



Training model.


In [59]:
from keras import callbacks
from keras.optimizers import RMSprop


rmsprop = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',    # compiling model......
              optimizer=rmsprop,
              metrics=['acc'])



In [60]:
model.fit(x_train, y_train,
          batch_size=64, #training NN having 20 epoch size with batch size of 64 , 2 hidden layes each consiting of 50 neurons....
          epochs=20,
          validation_data=(x_val, y_val))

Train on 45824 samples, validate on 11456 samples
Epoch 1/20
45824/45824 [==============================] - 76s 2ms/step - loss: 1.6348 - acc: 0.4940 - val_loss: 1.3416 - val_acc: 0.5948
Epoch 2/20
45824/45824 [==============================] - 72s 2ms/step - loss: 1.2518 - acc: 0.6053 - val_loss: 1.1362 - val_acc: 0.6440
Epoch 3/20
45824/45824 [==============================] - 72s 2ms/step - loss: 1.1469 - acc: 0.6333 - val_loss: 1.1334 - val_acc: 0.6452
Epoch 4/20
45824/45824 [==============================] - 71s 2ms/step - loss: 1.0867 - acc: 0.6524 - val_loss: 1.1226 - val_acc: 0.6460
Epoch 5/20
45824/45824 [==============================] - 72s 2ms/step - loss: 1.0572 - acc: 0.6605 - val_loss: 1.2196 - val_acc: 0.6027
Epoch 6/20
45824/45824 [==============================] - 71s 2ms/step - loss: 1.0518 - acc: 0.6646 - val_loss: 1.1615 - val_acc: 0.6594
Epoch 7/20
45824/45824 [==============================] - 70s 2ms/step - loss: 1.0332 - acc: 0.6685 - val_loss: 1.2398 - val_acc

In [61]:
model.evaluate(x_val, y_val) #evaluating model on validation dataset......

11456/11456 [==============================] - 3s 292us/step


[1.3649569644941297, 0.6474336592178771]

In [71]:
#all_words = set([word for document in all_text for word in document])

In [ ]:
#def get_featureset(document):
 #   return({word: (word in document) for word in all_words})
#processed_texts = [get_featureset(document) for document in all_text]

In [69]:
#print('Processing text dataset')

#raw_data = pd.read_csv('20news_raw.csv', encoding='latin-1')

### Converting string labels into numbers for neural-network
#labels_index = {}
#$for idx, i in enumerate(raw_data.categories.unique()):
#    labels_index[i] = idx


#print('Found %s texts.' % len(raw_data.texts))
#print('Found %d categories' %len(labels_index))

#print('Column Names', raw_data.columns)

Processing text dataset


FileNotFoundError: File b'20news_raw.csv' does not exist

In [64]:
x = TfidfVectorizer(stop_words='english', min_df=10, max_df=0.95)
x.fit(tf_idf_text)
tfidf = x.transform(tf_idf_text)
print(tfidf.shape)

(57280, 6032)


In [65]:
from keras.utils.np_utils import to_categorical
import numpy as np
#labels = [labels_index[label] for label in raw_data.categories]
#labels = to_categorical(np.asarray(labels))

In [66]:
model = Sequential()
model.add(Dense(500, input_dim=tfidf.shape[1]))
model.add(Dense(len(labels_index), activation='softmax'))

In [68]:
from keras.optimizers import SGD
sgd = SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(tfidf.todense(), labels, validation_split=0.2, epochs=20)



Train on 45824 samples, validate on 11456 samples
Epoch 1/20
45824/45824 [==============================] - 192s 4ms/step - loss: 2.1197 - acc: 0.2699 - val_loss: 2.3792 - val_acc: 0.2066
Epoch 2/20
25664/45824 [===============>..............] - ETA: 11:26 - loss: 1.9421 - acc: 0.3287

KeyboardInterrupt: 

### Using sklearn's neural network

In [69]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(500,), max_iter=5)



In [ ]:
clf.fit(tfidf.todense(), labels)

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(clf.predict(tfidf.todense()), labels)

0.85027399681810145